In [32]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

filename = "math sample for API testing.csv"

fields ={'Internal_ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [94]:
def process_file(filename, fields):
    process_fields = fields.keys()
    data = []
    holding = []
#    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()

        for line in reader:
            holding_test = 'FALSE'
            row = {}
            strand_dict = {}
            strand_list = []
            InternalID = int(line['Internal ID'])
            #row['Strand'] = line['Strand'].split("; ")
            Strand = line['Strand'].split("; ")
            for strand in Strand:
                strand_sub_dict = {}
                StrandGUID = get_GUID_lite(strand, content_focus_df)
                strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID}
                strand_list.append(strand_sub_dict)
            #strand_dict = {"Strand": strand_list} 
            row = {"Internal_ID": InternalID, "Organizers" : strand_list}        
            #row['Substrand'] = line['Sub-Strand'].split("; ")
            
#            for key,value in fields.items():
#                row[value] = (line[key]).split("; ")
#                #If strand, substrand, Content Focus or Numbers field in raw data has ";", then add to holding list
#                if (value == "Strand" or value == "Substrand" or value == "Content_Focus" or value == "Number") and (re.search(';', line[key])):
#                    holding_test = 'TRUE'
#            if holding_test == 'TRUE':
#                holding.append(row) 
#            else:  
            data.append(row)
    #Process holding list
    #Do the row have multiple values in strand both and substrand? If yes, then consult table_of_values to figure out what should get 
    #paired together
    #pprint.pprint(holding)
    return data

In [95]:
data = process_file(filename, fields)
pprint.pprint(data)

[{'Internal_ID': 333179,
  'Organizers': [{'Strand': 'Geometry and Measurement',
                  'StrandGUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA'},
                 {'Strand': 'Algebra',
                  'StrandGUID': 'D66A43AC-8BFB-11E6-921B-89CC29C466BA'}]},
 {'Internal_ID': 333010,
  'Organizers': [{'Strand': 'Geometry and Measurement',
                  'StrandGUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA'},
                 {'Strand': 'Algebra',
                  'StrandGUID': 'D66A43AC-8BFB-11E6-921B-89CC29C466BA'}]},
 {'Internal_ID': 333009,
  'Organizers': [{'Strand': 'Geometry and Measurement',
                  'StrandGUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA'},
                 {'Strand': 'Algebra',
                  'StrandGUID': 'D66A43AC-8BFB-11E6-921B-89CC29C466BA'}]},
 {'Internal_ID': 331547,
  'Organizers': [{'Strand': 'Algebra',
                  'StrandGUID': 'D66A43AC-8BFB-11E6-921B-89CC29C466BA'},
                 {'Strand': 'Geometry and Measurement',
   

In [75]:
pprint.pprint(data)

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]


In [11]:
#Make every internal ID of int data type "Internal ID"

In [12]:
#load GUID file

In [ ]:
#problem to solve for looking up GUID: when strand/substrand gives context to component, 
#if semi-colon separated, then will have to allow for non-matches

In [61]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [62]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [63]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
numbers_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")

In [72]:

def get_GUID_lite(tag, df): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        GUID = "no matches found"
    else: GUID = "resolve multiple values"
    return GUID

In [65]:
print steps_df

       Subject       Title                             Unique ID
0  Mathematics   multistep  F022E4A0-97E2-11E6-9F49-DB8029C466BA
1  Mathematics    two step  F024BB90-97E2-11E6-9F49-DB8029C466BA
2  Mathematics  three step  F02594A2-97E2-11E6-9F49-DB8029C466BA
3  Mathematics    one step  F0267624-97E2-11E6-9F49-DB8029C466BA


In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument


In [33]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
            {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
        {Units: [Units1, ...]}
        {Steps: Steps}
        {Context: Context}
        {SIG Stem: [Sig Stem1, ...]}
        {SIG Options: [SIG Options1, ...]}
        {Additional Modifiers: [AM1, ...]}
}

'''

'\nData structure\n\n[Internal ID: value]\n    {Organizers: [\n        {Strand: [ {Strand: Strand_value,\n                   Strand_GUID: GUID,\n                   Substrand: [{Substrand: Substrand_value,\n                                Substrand_GUID: GUID,\n                                Content_Focus: [{Content_Focus: Content_Focus_value,                       \n                                                Content_Focus_GUID: GUID,\n                                                Component: [{Component: Component_value,\n                                                              Component_GUID: GUID}]}]}]}]\n        {Descriptors: {Number: [N1, N2, ...]}\n            {Digits: [D1, D2, ...]}\n            {Shape: [Shape1, ...]}\n        {Units: [Units1, ...]}\n        {Steps: Steps}\n        {Context: Context}\n        {SIG Stem: [Sig Stem1, ...]}\n        {SIG Options: [SIG Options1, ...]}\n        {Additional Modifiers: [AM1, ...]}\n\n\n'